In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate mediapy triton scipy ftfy spacy==3.4.4

In [ ]:
# The xformers package is mandatory to be able to create several 768x768 images.
%pip install -q xformers==0.0.16rc425

# Selecionar modelo
Aqui você pode escolher entre um dos modelos disponíveis ou adcionar outro disponível no site hugging face. Para isso copie o nome do repositorio para o campo abaixo. Se alterar o modelo durante a exucução é necessário rodar novamente a partir deste bloco. Os modelos disponiveis aqui são:
- stabilityai/stable-diffusion-2-1-base: Modelo stable diffusion base v2.1
- stabilityai/stable-diffusion-2-1: Modelo stable diffusion base v2.1
- dreamlike-art/dreamlike-photoreal-2.0: Modelo para imagens realistas
- Ojimi/anime-kawai-diffusion: Estilo anime genérico
- DGSpitzer/Cyberpunk-Anime-Diffusion: Modelo treinado no anime Cyberpunk: Edgerunners
- danbrown/A-to-Zovya-RPG-v1-5: Modelo treinado em estilo fantasia

In [ ]:

model_id = "dreamlike-art/dreamlike-photoreal-2.0" #@param ["stabilityai/stable-diffusion-2-1-base", "stabilityai/stable-diffusion-2-1", "dreamlike-art/dreamlike-photoreal-2.0", "Ojimi/anime-kawai-diffusion","DGSpitzer/Cyberpunk-Anime-Diffusion","danbrown/A-to-Zovya-RPG-v1-5"] {allow-input: true}
# model_id = "stabilityai/stable-diffusion-2-1-base"  # modelo stable diffusion base v2.1
# model_id = "stabilityai/stable-diffusion-2-1"       # modelo stable diffusion base v2.1
# model_id = "dreamlike-art/dreamlike-photoreal-2.0"  # modelo para imagens realistas
# model_id = "Ojimi/anime-kawai-diffusion"            # estilo anime genérico
# model_id = "DGSpitzer/Cyberpunk-Anime-Diffusion"    # modelo treinado em Cyberpunk: Edgerunners
# model_id = "danbrown/A-to-Zovya-RPG-v1-5"             # modelo treinado para artes de rpg

In [ ]:
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler, EulerDiscreteScheduler, DPMSolverMultistepScheduler

scheduler = None
# scheduler = PNDMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = LMSDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler")

In [ ]:
import mediapy as media
import torch
from diffusers import StableDiffusionPipeline
import random

device = "cuda"

if model_id.startswith("stabilityai/"):
  model_revision = "fp16"
else:
  model_revision = None

if scheduler is None:
  pipe = StableDiffusionPipeline.from_pretrained(
      model_id,
      torch_dtype=torch.float16,
      revision=model_revision,
      )  
else:
  pipe = StableDiffusionPipeline.from_pretrained(
      model_id,
      scheduler=scheduler,
      torch_dtype=torch.float16,
      revision=model_revision,
      )

pipe = pipe.to(device)
pipe.enable_xformers_memory_efficient_attention()

if model_id.endswith('-base'):
  image_length = 512
else:
  image_length = 768


In [ ]:
#@title Prompt
#@markdown O que se deseja na imagem:
prompt = "\"A world of fire, atmospheric, hyper realistic, 8k, epic composition, cinematic, octane render, artstation landscape\"" #@param {type:"string"}
#@markdown O que deve se evitar na imagem:
negative_prompt = "blurry, low resolution, bad composition, weird faces, weird hands" #@param {type:"string"}

#@markdown Para resolucoes altas diminuir o numero de imagens (Resolucao alta nao recomendada se rodando pelo google collab. Pode causar bloqueio de acesso a gpu)
num_images = 2 #@param {type: "integer"}

image_width = 512 #@param {type:"integer"}
image_height = 512 #@param {type:"integer"}
num_steps = 50 #@param {type:"integer"}
guidance_scale = 9 #@param {type:"slider", min:0, max:10, step:1}
#@markdown Seed aleatoria para valores menor que 0
seed = 1628444291 #@param {type:"integer"}
remove_safety = False #@param {type:"boolean"}
#@markdown salva imagens como output_0,..,output_n
save_output = True #@param {type: "boolean"} 
if remove_safety:
  pipe.safety_checker = None
else:
  negative_prompt = negative_prompt.join("nude, naked, gore, nsfw")

if seed < 0: seed = random.randint(0, 2147483647)
images = pipe(
    prompt,
    height = image_height,
    width = image_width,
    num_inference_steps = num_steps,
    guidance_scale = guidance_scale,
    num_images_per_prompt = num_images,
    negative_prompt = negative_prompt,
    generator = torch.Generator("cuda").manual_seed(seed)
    ).images
    
media.show_images(images)
display(f"Seed: {seed}")
if save_output:
  for i, image in enumerate(images):
    image.save(f"output_{i}.jpg")
